# Script deployment boilerplate

Deploying assets (Functions, Models, Scripts) consists mainly in these main steps:

- Preparing the environment: creating a deployment space and generating WML credentials. 


- Publishing to WML: 

   1- Specifying the asset's Metadata (Name and Software specifications). 
   
   2- Then publishing it as an asset in WML: ``` command wml_client.repository.store_model ```
   
   
- Deploying to the Space:

   1- Getting the published asset ID: published_model_id
   
   2- Specifying some Metadata (name and the type (online/batch): metadata
   
   3- Deploying the asset with ``` wml_client.deployments.create(published_model_id, meta_props = metadata) ```
   
   
- For Scripts: **Define the job to run the script.**

In the example below, we demonstrate the **Batch** deployment of a Script. The script takes a .csv file as input and produces a .zip file as output, after some processing operations (rescaling mainly).


## Preparing the environment:

In [7]:
import wget
import os 
import pandas as pd 
import numpy as np 
from ibm_watson_machine_learning import APIClient

In [158]:
# !rm "transform_pred_script.py"

In [159]:
## Downloading a script example. 
url = "https://github.com/IBMDataScience/sample-notebooks/raw/master/Files/transform_pred_script.py"
if not os.path.isfile(url):
    wget.download(url)

In [195]:
# !ls

In [8]:
url = 'https://dse-cpd35-cluster2.datascienceelite.com'

wml_credentials = {
    "username": 'xxxx',
    "password": "xxxx", 
    "url": url, 
    "instance_id": "openshift", 
    "version": '3.5'
}

client = APIClient(wml_credentials)

In [9]:
## Connecting the WML to the deployment space
space_id = "8d2ad13c-6f02-4b12-9ffe-65642cf200eb"
client.set.default_space(space_id)

'SUCCESS'

## Publishing to WML:

In [162]:
## Creating the script's metadata 
## Here, I am specifying the language and version used for the script

soft_spec_uid = client.software_specifications.get_uid_by_name('default_py3.7')
script_meta_props = {
    client.script.ConfigurationMetaNames.NAME: 'My_deployed_script',
    client.script.ConfigurationMetaNames.SOFTWARE_SPEC_UID: soft_spec_uid   
}

## Then store the script in WML
script_meta = client.script.store(script_meta_props, "transform_pred_script.py")

Creating Script asset...
SUCCESS


To get the complete list of software specifications, run: (for more details: https://www.ibm.com/support/producthub/icpdata/docs/content/SSQNUZ_latest/wsj/wmls/wmls-deploy-python-types.html)

In [10]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
scikit-learn_0.22-py3.6        154010fa-5b3b-4ac1-82af-4d5ee5abbc85  base
default_r3.6                   1b70aec3-ab34-4b87-8aa0-a4a3c8296a36  base
tensorflow_1.15-py3.6          2b73a275-7cbf-420b-a912-eae7f436e0bc  base
pytorch_1.2-py3.6              2c8ef57d-2687-4b7d-acce-01f94976dac1  base
spark-mllib_2.3                2e51f70

In [42]:
print(script_meta)

{'metadata': {'space_id': '8d2ad13c-6f02-4b12-9ffe-65642cf200eb', 'name': 'My_deployed_script', 'guid': 'c3829b9f-f751-47be-966a-1e07f3cb16c2', 'href': '/v2/assets/c3829b9f-f751-47be-966a-1e07f3cb16c2?space_id=8d2ad13c-6f02-4b12-9ffe-65642cf200eb', 'asset_type': 'script', 'created_at': '2021-06-02T22:40:51Z', 'last_updated_at': '2021-06-02T22:40:51Z', 'description': ''}, 'entity': {'script': {'language': {'name': 'python3'}, 'software_spec': {'base_id': 'e4429883-c883-42b6-87a8-f419d64088cd'}}}}


In [163]:
script_uid = client.script.get_uid(script_meta)
print(script_uid)

27ae8d20-524e-4841-8115-3617e9e7bfb5


## Script deployment

Now it's time to deploy the script. The script is deployed as a batch (meaning, it processes a file as input as produces an output)

In [164]:
script_deploy_meta = {
    client.deployments.ConfigurationMetaNames.NAME: 'My_script_deployment', 
    client.deployments.ConfigurationMetaNames.BATCH: {}, 
    client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: {'name': 'S'}
}

script_deployment = client.deployments.create(script_uid, script_deploy_meta)



#######################################################################################

Synchronous deployment creation for uid: '27ae8d20-524e-4841-8115-3617e9e7bfb5' started

#######################################################################################


ready.


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='83e8f1bd-45b8-4ef6-b540-79d49f83358f'
------------------------------------------------------------------------------------------------




In [165]:
client.deployments.list()

------------------------------------  -------------------------  -----  ------------------------
GUID                                  NAME                       STATE  CREATED
83e8f1bd-45b8-4ef6-b540-79d49f83358f  My_script_deployment       ready  2021-06-03T16:19:59.952Z
338efb5e-d00b-4d77-bc47-9c92eab92d09  mnist_function_deployment  ready  2021-06-01T21:57:21.210Z
4e66b8e3-afb8-480c-a132-18a52d61dfaf  mnist_function_deployment  ready  2021-06-01T21:36:53.562Z
b2b25a81-7fcc-4a9e-9e6d-ebdbaf72261c  sample_model_deployment    ready  2021-06-01T20:09:00.872Z
------------------------------------  -------------------------  -----  ------------------------


## Loading a data sample for the script

In [172]:
url = "https://raw.githubusercontent.com/sarahboufelja/sample-notebooks/master/car_results.csv"
if not os.path.isfile(url):
    wget.download(url)

In [173]:
!ls

car_results.csv			   mnist-tf-hpo-saved-model.tar.gz  user.log
imports-85.data			   results_scripts_output.zip
mnist-html-canvas-image-data.json  transform_pred_script.py


The input file can be added in two ways:

1- Manually add the file to the Project's asset and promote it so it can be published in WML. From there you can either deploy it manually from the ***GUI*** or programatically from Jupyter notebook. 

2- Define the asset's ID and the asset's HREF form this Jupyter Notebook (as demonstrated below). For this, you will need to use thiis command: ``` client.data_assets.get_href(asset_details) ``` where ``` asset_details ``` are the WML asset definition. 


In [178]:
## create the asset HREF for the payload, the names are chosen to be coherent with the script. 
asset_details = client.data_assets.create(name= "car_results.csv", file_path="car_results.csv")
asset_details

Creating data asset...
SUCCESS


{'metadata': {'space_id': '8d2ad13c-6f02-4b12-9ffe-65642cf200eb',
  'guid': '8379c9da-3713-4ea8-b27e-aec0e180ea47',
  'href': '/v2/assets/8379c9da-3713-4ea8-b27e-aec0e180ea47?space_id=8d2ad13c-6f02-4b12-9ffe-65642cf200eb',
  'asset_type': 'data_asset',
  'created_at': '2021-06-03T16:31:18Z',
  'last_updated_at': '2021-06-03T16:31:18Z'},
 'entity': {'data_asset': {'mime_type': 'text/csv'}}}

In [179]:
asset_id = client.data_assets.get_uid(asset_details)
print(asset_id)

8379c9da-3713-4ea8-b27e-aec0e180ea47


In [180]:
asset_href = client.data_assets.get_href(asset_details)
print(asset_href)

/v2/assets/8379c9da-3713-4ea8-b27e-aec0e180ea47?space_id=8d2ad13c-6f02-4b12-9ffe-65642cf200eb


In [181]:
client.data_assets.list()

---------------  ----------  -----  ------------------------------------
NAME             ASSET_TYPE  SIZE   ASSET_ID
input_data       data_asset  25936  11c6ad86-112e-44ec-8a46-221fa504090a
car_results.csv  data_asset  242    8379c9da-3713-4ea8-b27e-aec0e180ea47
---------------  ----------  -----  ------------------------------------


## Define the job to run the script

In [182]:
from pprint import pprint

script_payload = {
    
    client.deployments.ScoringMetaNames.INPUT_DATA_REFERENCES: [{
        "name": "script_ref_inputs", 
        "type": "data_asset", 
        "connection": {}, 
        "location": {
            "href": asset_href
        }, 
    }], 
    
    client.deployments.ScoringMetaNames.OUTPUT_DATA_REFERENCE: {
        "type": "data_asset", 
        "connection": {}, 
        "location": {
            "name": "result_script_output.zip", 
            "descripton": "script_results"
        }
    }
}

pprint(script_payload)

{'input_data_references': [{'connection': {},
                            'location': {'href': '/v2/assets/8379c9da-3713-4ea8-b27e-aec0e180ea47?space_id=8d2ad13c-6f02-4b12-9ffe-65642cf200eb'},
                            'name': 'script_ref_inputs',
                            'type': 'data_asset'}],
 'output_data_reference': {'connection': {},
                           'location': {'descripton': 'script_results',
                                        'name': 'result_script_output.zip'},
                           'type': 'data_asset'}}


In [183]:
## Publish the script in WML

script_deployment_uid = client.deployments.get_uid(script_deployment)
print(script_deployment_uid)

job = client.deployments.create_job(deployment_id = script_deployment_uid, meta_props = script_payload)
script_job_uid = client.deployments.get_job_uid(job)
print(script_job_uid)

83e8f1bd-45b8-4ef6-b540-79d49f83358f
cc88b983-89d5-42cc-a360-844926c2ac9a


### Running the Job

In [184]:
## A helper function to return the job status 

def run_async_job(client, job_uid):
    import time
    while True:
        job_status = client.deployments.get_job_status(job_uid)
        print(job_status)
        state = job_status['state']
        if state == 'completed' or 'failed' in state:
            return client.deployments.get_job_details(job_uid)
        time.sleep(4)

In [185]:
## Run the job once

job_details = run_async_job(client, script_job_uid)
pprint(job_details)

{'completed_at': '2021-06-03T16:36:51.367523Z', 'running_at': '2021-06-03T16:36:48.743586Z', 'state': 'completed'}
{'entity': {'deployment': {'id': '83e8f1bd-45b8-4ef6-b540-79d49f83358f'},
            'platform_job': {'job_id': 'd10ac41b-672d-4871-aafd-47815e0e473d',
                             'run_id': '8df9c37a-3d3e-4af9-aad3-308e656314b3'},
            'scoring': {'input_data_references': [{'connection': {},
                                                   'location': {'href': '/v2/assets/8379c9da-3713-4ea8-b27e-aec0e180ea47?space_id=8d2ad13c-6f02-4b12-9ffe-65642cf200eb'},
                                                   'type': 'data_asset'}],
                        'output_data_reference': {'connection': {},
                                                  'location': {'descripton': 'script_results',
                                                               'href': '/v2/assets/c02618e7-5fa3-4330-9e87-0eee6b50c7b5?space_id=8d2ad13c-6f02-4b12-9ffe-65642cf200eb',
       

In [186]:
client.data_assets.list()

------------------------  ----------  -----  ------------------------------------
NAME                      ASSET_TYPE  SIZE   ASSET_ID
car_results.csv           data_asset  242    8379c9da-3713-4ea8-b27e-aec0e180ea47
input_data                data_asset  25936  11c6ad86-112e-44ec-8a46-221fa504090a
result_script_output.zip  data_asset  228    c02618e7-5fa3-4330-9e87-0eee6b50c7b5
------------------------  ----------  -----  ------------------------------------


As you can see from the above command, our file "result_script_output.zip" has been created successfully

## Download results.zip file

In [187]:
import re
results = job_details['entity']['scoring']['output_data_reference']['location']['href']
results = re.split('[?/]', results)[3]
print(results)
client.data_assets.download(results, "results_scripts_output.zip")

c02618e7-5fa3-4330-9e87-0eee6b50c7b5
Successfully saved data asset content to file: 'results_scripts_output.zip'


'/home/wsuser/work/results_scripts_output.zip'

In [197]:
#!unzip -p results_scripts_output.zip

In [190]:
!ls

car_results.csv			   mnist-tf-hpo-saved-model.tar.gz  user.log
imports-85.data			   results_scripts_output.zip
mnist-html-canvas-image-data.json  transform_pred_script.py
